# Marabou's Split & Conquer Guide

This Notebook illustrates properties of the Split & Conquer (S&C) algorithm in Marabou, provides an overview of algorithm parameters, shows their impact on algorithm performance and offers some rules of thumb on how to choose parameter values for problems of unknown difficulty.

## About Split & Conquer algorithm
The algorithm is build on top of Marabou solver for neural network verification(NNV). Scalability is one of the chief hurdles for NNV. S&C algorithm is an effort-based algorithm, which dynamically decomposes queries which exceed the effort limit. 

Effort is inpractice specified in the form of timeout - initially *t_0*. Whenever a query exceeds its timeout, the S&C algorithm will split it into a number of new queries. The number of splits *n* is another parameter of the algorithm, and will result in *2^n* new queries created. This new generation of queries will be solved with an increased timeout, which is specified through the timeout factor *f*. Each of the queries will in turn be solved with the new timout and this process is repeated, until either:
1. a subquery is shown to be **satisfiable**
2. or **all** the generated sub-queries are **unsatisfiable**.

Here is an illustration of the S&C algorithm execution with 4 parallel workers:
![Execution of Split & Conquer algorithm](img/snc.png)

### How to split the query?
Queries can be split in various way - the two that have shown the most success are:

1. **input range splitting** for networks with few inputs and 
2. **early ReLU phase splitting** for perception networks.

Both approaches still require to choose which particular input or ReLU to split on. Marabou implements several heuristics for this choice. The crucial property for parallelization is that the resulting queries are of similar difficulty, which in terms of search space means that the splits are *balanced*. The metric to estimate balance we call **polarity** and is the default choice for S&C.

A crucial point is that the above splitting strategies generate queries whose disjucntion is equivalent to the original query. This is essential for completeness of the overall algorithm and trustworthiness of reported results.

## Getting a feel for S&C parameters

In this section we will run a series of examples to show how different parameter values affect execution.

### First, run a sequential problem

In [ ]:
# Supress TF deprecation warnings
import warnings
warnings.filterwarnings('ignore')

import sys

# Path to Marabou folder if you did not export it
# sys.path.append('/PATH/TO/Marabou')


from maraboupy import Marabou



In [ ]:
# Choose an input file that takes about 30seconds to solve 
# Network files are sorted in increasing order of difficulty
#nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_2_6.nnet"
#nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_2_5.nnet"
nnetFile = "nnet/acasxu/ACASXU_experimental_v2a_5_2.nnet"

# Load the network from NNet file
net = Marabou.read_nnet(nnetFile)

# Encode property 3 - see properties/acas_property3.txt
lower_bounds = [-0.3035311561, -0.0095492966, 0.4933803236, 0.3,  0.3]
upper_bounds = [ -0.2985528119, 0.0095492966, 0.5, 0.5, 0.5]

for idx in range(len(net.inputVars[0])):
    net.setLowerBound(net.inputVars[0][idx], lower_bounds[idx])
    net.setUpperBound(net.inputVars[0][idx], upper_bounds[idx])

net.addInequality(net.outputVars[0], [+1.0,-1.0, 0.0, 0.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0,-1.0, 0.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0, 0.0,-1.0, 0.0], 0.0)
net.addInequality(net.outputVars[0], [+1.0, 0.0, 0.0, 0.0,-1.0], 0.0)


In [ ]:
%%time
defaultOpts = Marabou.createOptions()
vals1, stats1 = net.solve(options=defaultOpts)

The query is reported unsat and we note the wall clock time as a baseline to solve this problem using Marabou. 

For the purposes of this demo choose a network among commented options that takes ~15 seconds on your machine.

Run locally, solving Property 3 on ACASXU_experimental_v2a_5_2.nnet takes 14s.

### Run a problem using S&C
To run a problem in S&C mode, add *snc=True* to the Marabou.createOptions call. 

Since only S&C moded is enable and no other S&C parameters have been set, the algorithm is executed **sequentially**.

In [ ]:
%%time
sequentialSnC = Marabou.createOptions(snc=True)
vals1, stats1 = net.solve(options=sequentialSnC)

The query is again found unsat, which we know from the previous call to Marabou.

Marabou in SnC mode outputs lines of the following format on the command line/Jypyter console:

Worker ID : Query ID SOLVING_STATUS, number of remaining tasks.

> Worker 0: Query 1 TIMEOUT, 1 tasks remaining

The algorithm is executed sequentially so Worker 0 will feature on all the lines. 

Query 1 denotes the original query and SOLVING_STATUS can be sat/unsat/TIMEOUT. 

Here, Query 1 TIMEOUTS with default timeout. We refer to Query 1 as the intial query or the initial generation.

The default number of splits is 2, so upon TIMEOUT S&C will generate 4 (2^2) subqueries. 

Each query has an ID that shows its location in the search space. Query 1-3 indicates that is the 3rd split of Query 1. We refer to Queries 1-X as first generation. Each dash in the name will indicate another generation.

> Worker 0: Query 1-1 unsat, 4 tasks remaining  
  Worker 0: Query 1-2 unsat, 3 tasks remaining  
  Worker 0: Query 1-3 unsat, 2 tasks remaining  
  Worker 0: Query 1-4 unsat, 1 tasks remaining  

We can see that after splitting and timeout increase, all 4 subqueries are found to be unsat which which we expect since Query 1 is unsat.

Run locally sequential S&C took 17s, which is a decrease in performance compared to 14s Marabou took to solve the same problem. This is expected as SnC introduces overhead with each intermediate subquery that times out.

### Run a problem using S&C in parallel
To execute SnC in parallel add *numWorkers = 4* in the Marabou.createOptions(). 

To make the number of splits explicit we will add *onlineSplits = 2* to the options. S&C will perform 2^onlineSplits after every TIMEOUT. Since the default value is also 2 there will be no change in the number of subqueries generated compared to the sequential execution. 

In [ ]:
%%time
SnC4workers = Marabou.createOptions(snc=True,
                                    numWorkers=4,
                                    onlineSplits=2
                                   )
vals1, stats1 = net.solve(options=SnC4workers)

Running locally with 4 workers this call took 10s of wall clock time which less than both Marabou and sequential S&C call. The total CPU time is the same for sequential and parallel S&C calls, the CPU resources are utilized more effectively.

If we look at the S&C output:

> Worker 3: Query 1 TIMEOUT, 1 tasks remaining  
  Worker 1: Query 1-2 unsat, 4 tasks remaining  
  Worker 3: Query 1-1 unsat, 3 tasks remaining  
  Worker 0: Query 1-4 unsat, 2 tasks remaining  
  Worker 2: Query 1-3 unsat, 1 tasks remaining

we can notice that now there are 4 different workers featuring in the output.

If we think for a moment, we will realize that Workers 0,1 and 2 were idling while Query 1 was solved. We will address this in the next section

### Run parallel S&C with initial splits
Rather then letting workers idle during the first query, a query can be immediately split to match the number of workers, skipping Query 1 and jumping directly to generation 1 of the previous examples. To achieve this we add *initialSplits=2* to the options.


In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineSplits=2,
                                     initialSplits=2)
vals1, stats1 = net.solve(options=SnC4workers)

This took 4.75s of wall clock time on my local machine, beating all previous wall clock runtimes. 

Since S&C is not waiting on the initial query to timeout, the total time is 11s. This is less than total times of the baseline Marabou call and previous S&C calls.

Looking at the output:
> Worker 1: Query 2 unsat, 4 tasks remaining  
  Worker 0: Query 1 unsat, 3 tasks remaining  
  Worker 2: Query 4 unsat, 2 tasks remaining  
  Worker 3: Query 3 unsat, 1 tasks remaining

We can see that the queries are treated as initial generation, meaning that their IDs do not contain dashes. Implicitly, (Query) 0- is ommited. 

With these options we can solve the problem in one generation. Now, let's take a look at how timeout and timeout factor affect the behavior of S&C algorithm.

### S7C: Choosing initial timeout

To set the initial timeout add *initialTimeout=3* to the Marabou options. This will set the timeout to 3 seconds which is less than the default of 5 seconds.

In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineSplits=2,
                                     initialSplits=2,
                                     initialTimeout=3)
vals1, stats1 = net.solve(options=SnC4workers)

This call took 5s on my machine, which is just slightly longer than the pervious call of 4.75s.

Let's look at the output of S&C:
> Worker 3: Query 2 unsat, 4 tasks remaining  
  Worker 0: Query 1 unsat, 3 tasks remaining  
  Worker 1: Query 4 unsat, 2 tasks remaining  
  Worker 2: Query 3 TIMEOUT, 1 tasks remaining  
  Worker 0: Query 3-3 unsat, 4 tasks remaining  
  Worker 2: Query 3-1 unsat, 3 tasks remaining  
  Worker 1: Query 3-4 unsat, 2 tasks remaining  
  Worker 3: Query 3-2 unsat, 1 tasks remaining
  
We can see that Query 3 timed out resulting in a new generation of problems to be solved, however they were quickly solved. 

Let's take a look at what happens when we decrease the initial timeout further to 1s.

In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineSplits=2,
                                     initialSplits=2,
                                     initialTimeout=1)
vals1, stats1 = net.solve(options=SnC4workers)

This call takes 4s which is faster than 4.75s obtained with the default timeout.

Looking at the output of S&C:
> Worker 0: Query 2 unsat, 4 tasks remaining  
  Worker 2: Query 1 TIMEOUT, 3 tasks remaining  
  Worker 1: Query 4 TIMEOUT, 6 tasks remaining  
  Worker 3: Query 3 TIMEOUT, 9 tasks remaining  
  Worker 1: Query 1-2 unsat, 12 tasks remaining  
  Worker 1: Query 4-1 unsat, 11 tasks remaining  
  Worker 3: Query 1-4 unsat, 10 tasks remaining  
  Worker 0: Query 1-3 unsat, 9 tasks remaining  
  Worker 1: Query 4-2 unsat, 8 tasks remaining  
  Worker 0: Query 4-4 unsat, 7 tasks remaining  
  Worker 2: Query 1-1 unsat, 6 tasks remaining  
  Worker 2: Query 3-3 unsat, 5 tasks remaining  
  Worker 1: Query 3-1 unsat, 4 tasks remaining  
  Worker 3: Query 4-3 unsat, 3 tasks remaining  
  Worker 2: Query 3-4 unsat, 2 tasks remaining  
  Worker 0: Query 3-2 unsat, 1 tasks remaining
  
We can see that most of the initial generation times out, but that overall solving time improves. This suggests that **aggessive splitting might be a good strategy for this problem**. However, rather than wasting time by letting most of the initial generation time out, let's increase the number of initial splits to 4 (which results in 16 problems) and see what happens.


In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=4,
                                     onlineSplits=2,
                                     initialSplits=4,
                                     initialTimeout=5)
vals1, stats1 = net.solve(options=SnC4workers)

This call took 1.95s of wall clock time and 5.81s of CPU time. Which reduces both wall-clock and total time by nearly 50%.

So a rule of thumb is that if initial generations are timing out, either:
1. **aggressively split using initialSplits** (regardless of the number of workers, they will be properly queued)
2. **increase the timeout until some intial queries a solved effectively**, in particular when dealing with large networks (>1k nodes) this will be necessary. 

This is just one of the timeout parameters, the other is timeout factor which we will take a look at in the next section.

### S&C: Choosing timeout factor
The timeout factor indicates how much extra effort we want to spend on the next generation. For completeness purposes the factor needs to be strictly greater than 1, although with aggressive splitting that may be unnecessary.

To simulate a more challenging query, let's reduce the number of workers to 2 and set initial and online divides and timeout to 1. We'll set the *timeoutFactor=1.25*, i.e. each generation gets 25% more time than the previous one.

In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=2,
                                     onlineSplits=1,
                                     initialSplits=1,
                                     initialTimeout=1,
                                     timeoutFactor=1.25)
vals1, stats1 = net.solve(options=SnC4workers)

This call takes 10s of wall clock time. 

Looking at the S&C ouput:

> Worker 1: Query 2 TIMEOUT, 2 tasks remaining  
  Worker 0: Query 1 TIMEOUT, 3 tasks remaining  
  Worker 1: Query 2-1 TIMEOUT, 4 tasks remaining
  Worker 0: Query 2-2 TIMEOUT, 5 tasks remaining
  Worker 0: Query 1-2 unsat, 6 tasks remaining  
  Worker 0: Query 2-1-1 unsat, 5 tasks remaining  
  Worker 1: Query 1-1 TIMEOUT, 4 tasks remaining  
  Worker 1: Query 2-2-1 unsat, 5 tasks remaining  
  Worker 0: Query 2-1-2 unsat, 4 tasks remaining  
  Worker 1: Query 2-2-2 unsat, 3 tasks remaining  
  Worker 0: Query 1-1-1 unsat, 2 tasks remaining  
  Worker 1: Query 1-1-2 unsat, 1 tasks remaining

For the next attempt lets increase the timeoutFactor to 3, and observe the results.

In [ ]:
%%time
SnC4workers = Marabou.createOptions( snc=True,
                                     numWorkers=2,
                                     onlineSplits=1,
                                     initialSplits=1,
                                     initialTimeout=1,
                                     timeoutFactor=3)
vals1, stats1 = net.solve(options=SnC4workers)

This call took 9.84s which is barely less than 10s of the previous call. 

We can see that it needed to solve fewer queries:
> Worker 1: Query 1 TIMEOUT, 2 tasks remaining  
  Worker 0: Query 2 TIMEOUT, 3 tasks remaining  
  Worker 0: Query 1-2 unsat, 4 tasks remaining  
  Worker 1: Query 1-1 unsat, 3 tasks remaining  
  Worker 0: Query 2-1 TIMEOUT, 2 tasks remaining  
  Worker 1: Query 2-2 unsat, 3 tasks remaining  
  Worker 0: Query 2-1-1 unsat, 2 tasks remaining  
  Worker 1: Query 2-1-2 unsat, 1 tasks remaining  

But the time didn't improve much. What one needs to keep in mind when choosing the timeoutFactor is that wall clock solving time will be determined by the last generation generated. Each query in generation i takes T_0 * TimeoutFactor^i when it times out. 

One can look at a some initial set of solved queries of an incomplete solving run, use generation info to bucket them by solving time and use that information to choose intialSplits, timeout and timeoutFactor values in a way that will avoid unnecessary TIMEOUTs. 


This concludes this demo, and hopefully provides you with an idea on how to think about Split & Conquer parameters.
  